In [1]:
import multiprocessing
import numpy as np
import xarray as xr
import os
import pandas as pd
import netCDF4 as nc
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import linregress
from scipy.optimize import curve_fit
from pymer4.models import Lmer
import math
import seaborn as sns

In [ ]:
#生物量的季节性变化

In [3]:
type_data = pd.read_csv('./Site_igbp.csv', encoding='latin-1')
site_IGBP = type_data['igbp'].values
site_lat = type_data['lat'].values
site_ID =  type_data['site'].values

path = 'D:Data/Fluxnet/All_hourly_data/'

site = ['site']
igbp = ['igbp']
year_list = ['year']
RECO = [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'], ['10'], ['11'], ['12']]


for csv_file in os.listdir(path):
    if site_lat[site_ID==csv_file[4:10]][0] < 0:
        data = pd.read_csv(path+csv_file)
        year = (data['TIMESTAMP_START']/100000000).astype(int)
        month = np.round(data['TIMESTAMP_START']/1000000).astype(int)%100
        hour = (data['TIMESTAMP_START']%10000).astype(int)
        unique_values = np.unique(year)

        for i in range(len(unique_values)):
            Ti_0 = data['TA_F_MDS'].values
            RECOi_0 = data['RECO_NT_VUT_REF'].values
            T_QCi_0 = data['TA_F_MDS_QC'].values
        
            Reco_months = []
            
            Ti_1 = Ti_0[(Ti_0 !=-9999)]
            T_mean = np.mean(Ti_1)
            
            for j in range(12):
                if T_mean < 10:
                    RECOj = RECOi_0[(Ti_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.001) & (T_QCi_0==0) & (year==unique_values[i]) & (month==j+1) & (Ti_0>=9) & (Ti_0<11) & ((hour>=1900) | (hour<=700))]
                elif T_mean >= 10 and T_mean < 15:
                    RECOj = RECOi_0[(Ti_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.001) & (T_QCi_0==0) & (year==unique_values[i]) & (month==j+1) & (Ti_0>=14) & (Ti_0<16) & ((hour>=1900) | (hour<=700))]
                elif T_mean >= 15 and T_mean < 20:
                    RECOj = RECOi_0[(Ti_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.001) & (T_QCi_0==0) & (year==unique_values[i]) & (month==j+1) & (Ti_0>=19) & (Ti_0<21) & ((hour>=1900) | (hour<=700))]
                elif T_mean >= 20:
                    RECOj = RECOi_0[(Ti_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.001) & (T_QCi_0==0) & (year==unique_values[i]) & (month==j+1) & (Ti_0>=24) & (Ti_0<26) & ((hour>=1900) | (hour<=700))]
                if len(RECOj) >10:
                    RECO[j].append(np.mean(RECOj))
                else:
                    RECO[j].append(np.nan)
            
            site.append(csv_file[4:10])
            igbp.append(site_IGBP[site_ID==csv_file[4:10]][0])
            year_list.append(unique_values[i])
            
        result = np.array([site, igbp, year_list, RECO[0], RECO[1], RECO[2], RECO[3], RECO[4], RECO[5], RECO[6], RECO[7], RECO[8], RECO[9], RECO[10], RECO[11]])
        result_pd = pd.DataFrame(result.T)
        save_path = './Result1_intro/RECO_seasonal_change_south_nighttime.csv'
        result_pd.to_csv(save_path, index= 'a', header= False)

In [2]:
data = pd.read_csv('./Result1_intro/RECO_seasonal_change_south_nighttime.csv')
site = ['site']
igbp = ['igbp']
year_list = ['year']
RECO = [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'], ['10'], ['11'], ['12']]
m,n = data.shape

for i in range(m):
    RECO_all = []
    for j in range(12):
        RECO_all.append(data[str(j+1)][i])
    if np.sum(np.isnan(RECO_all)) <= 5:
        for j in range(12):
            RECO[j].append( RECO_all[j]/np.nanmean(RECO_all) )
        site.append(data['site'][i])
        igbp.append(data['igbp'][i])
        year_list.append(data['year'][i])
    result = np.array([site, igbp, year_list, RECO[0], RECO[1], RECO[2], RECO[3], RECO[4], RECO[5], RECO[6], RECO[7], RECO[8], RECO[9], RECO[10], RECO[11]])
    result_pd = pd.DataFrame(result.T)
    save_path = './Result1_intro/RECO_seasonal_change_normalized_south_nighttime.csv'
    result_pd.to_csv(save_path, index= 'a', header= False)

In [ ]:
#呼吸的月平均变化

In [2]:
type_data = pd.read_csv('./Site_igbp.csv', encoding='latin-1')
site_IGBP = type_data['igbp'].values
site_lat = type_data['lat'].values
site_ID =  type_data['site'].values

path = 'D:Data/Fluxnet/All_hourly_data/'

site = ['site']
igbp = ['igbp']
year_list = ['year']
RECO = [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'], ['10'], ['11'], ['12']]


for csv_file in os.listdir(path):
    if site_lat[site_ID==csv_file[4:10]][0] > -50:
        data = pd.read_csv(path+csv_file)
        year = (data['TIMESTAMP_START']/100000000).astype(int)
        month = np.round(data['TIMESTAMP_START']/1000000).astype(int)%100
        hour = (data['TIMESTAMP_START']%10000).astype(int)
        unique_values = np.unique(year)

        for i in range(len(unique_values)):
            Ti_0 = data['TA_F_MDS'].values
            RECOi_0 = data['RECO_NT_VUT_REF'].values
            T_QCi_0 = data['TA_F_MDS_QC'].values
        
            Reco_months = []
            
            Ti_1 = Ti_0[(Ti_0 !=-9999)]
            T_mean = np.mean(Ti_1)
            
            for j in range(12):
                RECOj = RECOi_0[(Ti_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.001) & (T_QCi_0==0) & (year==unique_values[i]) & (month==j+1) & ((hour>=1900) | (hour<=700))]
                if len(RECOj) >100:
                    RECO[j].append(np.mean(RECOj))
                else:
                    RECO[j].append(np.nan)
            
            site.append(csv_file[4:10])
            igbp.append(site_IGBP[site_ID==csv_file[4:10]][0])
            year_list.append(unique_values[i])
            
        result = np.array([site, igbp, year_list, RECO[0], RECO[1], RECO[2], RECO[3], RECO[4], RECO[5], RECO[6], RECO[7], RECO[8], RECO[9], RECO[10], RECO[11]])
        result_pd = pd.DataFrame(result.T)
        save_path = './Result1_intro/RECO_seasonal_change_monthly_mean_north_nighttime.csv'
        result_pd.to_csv(save_path, index= 'a', header= False)

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



In [13]:
path1 = './Result1_intro/RECO_seasonal_change_monthly_mean_south_nighttime.csv'
path2 = './Result1_intro/RECO_seasonal_change_south_nighttime.csv'

path3 = './Site_igbp.csv'
path4 = './Site_climate.csv'

data1 = pd.read_csv(path1)
data2 = pd.read_csv(path2)

data3 = pd.read_csv(path3)
data4 = pd.read_csv(path4)

m,n = data1.shape

site = ['site']
igbp = ['igbp']
climate = ['climate']
Delt_R_ref = ['delt_mean']
Delt_R_mean = ['delt_ref']

for i in range(m):
    R_1 = []
    R_2 = []
    for j in range(12):
        if np.isnan(data1[str(j+1)][i]) != 1 and np.isnan(data2[str(j+1)][i]) != 1:            
            R_1.append(data1[str(j+1)][i])
            R_2.append(data2[str(j+1)][i])

    if len(R_1)>=5 and len(np.unique(R_1))>=5 and len(np.unique(R_2))>=5# and data1['site'][i] in data4['site'].values:
        site.append(data1['site'][i])
        igbp.append(data3['igbp'].values[(data3['site'].values==data1['site'][i])][0])
        climate.append(data3['igbp'].values[(data4['site'].values==data1['site'][i])][0])
        Delt_R_ref.append(np.max(R_1)-np.min(R_1))
        Delt_R_mean.append(np.max(R_2)-np.min(R_2))
result = np.array([site, igbp, Delt_R_ref, Delt_R_mean])
result_pd = pd.DataFrame(result.T)
save_path = './Seasonal_variation_igbp_south.csv'
result_pd.to_csv(save_path, index= 'a', header= False)

In [ ]:
#生物量和温度的耦合对Q10的影响

In [ ]:
type_data = pd.read_csv('./Site_igbp.csv', encoding='latin-1')
site_IGBP = type_data['igbp'].values
site_lat = type_data['lat'].values
site_ID =  type_data['site'].values

path = 'D:Data/Fluxnet/All_hourly_data/'

site = ['site']
igbp = ['igbp']
year_list = ['year']
Q10 = ['Q10']
SE = ['SE']
p = ['pvalue']
cor = ['cor']

days = 10

for csv_file in os.listdir(path):
    if csv_file[0] == 'F': 
        if csv_file[31:33] =='HH':
            hours = 48
        else:
            hours = 24
    else:
        if csv_file[27:29] =='HH':
            hours = 48
        else:
            hours = 24

    data = pd.read_csv(path+csv_file)
    year = (data['TIMESTAMP_START']/100000000).astype(int)
    month = np.round(data['TIMESTAMP_START']/1000000).astype(int)%100
    hour = (data['TIMESTAMP_START']%10000).astype(int)
    unique_values = np.unique(year)
    
    for i in range(len(unique_values)):
        Ti_0 = data['TA_F_MDS'].values
        RECOi_0 = data['RECO_NT_VUT_REF'].values
        T_QCi_0 = data['TA_F_MDS_QC'].values
        Ti_1 = Ti_0[(Ti_0!=-9999) & (RECOi_0!=-9999) & (RECOi_0>0.001) & (T_QCi_0==0) & (year==unique_values[i]) & ((hour>=1900) | (hour<=700))]                        
        RECOi_1 = RECOi_0[(Ti_0!=-9999)& (RECOi_0!=-9999) & (RECOi_0>0.001) & (T_QCi_0==0) & (year==unique_values[i]) & ((hour>=1900) | (hour<=700))]
        Normal_Ti_1 = (Ti_1-10)/10
        ln_RECOi_1 = np.log(RECOi_1)
        

        #计算温度和生物量之间的耦合度
        Ti = Ti_0[(year==unique_values[i])]   
        houri = hour[(year==unique_values[i])]
        T_QCi = T_QCi_0[(year==unique_values[i])]
        RECOi = RECOi_0[(year==unique_values[i])]
        m = len(Ti)
        
        Reco_months = []
        T_months = []
        T_median = np.median(Ti_1)
        
        for j in range(int(m/hours/days)-1):
            Tj = Ti[i*days*hours:(j+1)*days*hours]
            T_QCj = T_QCi[i*days*hours:(j+1)*days*hours]
            RECOj = RECOi[i*days*hours:(j+1)*days*hours]
            hourj = houri[i*days*hours:(j+1)*days*hours]
            
            Tj_1 = Tj[(Tj!=-9999) & (RECOj!=-9999) & (RECOj>0.001) & (T_QCj==0) & ((hourj>=1900) | (hourj<=700))]
            RECOj_1 = RECOj[(Tj!=-9999) & (RECOj!=-9999) & (RECOj>0.001) & (T_QCj==0) & (Tj >= T_median-1) & (Tj < T_median+1) & ((hourj>=1900) | (hourj<=700))]

            if len(RECOj_1) >10 and len(Tj_1) >50:
                T_months.append(np.mean(Tj_1))
                Reco_months.append(np.mean(RECOj_1))
        
        if len(Reco_months) >=10:
            if hours ==48:
                t = 7500
            else:
                t = 3750
            if len(Normal_Ti_1) > t:
                y = ln_RECOi_1
                X = sm.add_constant(Normal_Ti_1)
                model = sm.OLS(y, X)
                results = model.fit()
                slope = results.params[1]
                slope_pvalue = results.pvalues[1]
                slope_standard_error = results.bse[1]
                
                site.append(csv_file[4:10])
                igbp.append(site_IGBP[site_ID==csv_file[4:10]][0])
                year_list.append(unique_values[i])
                Q10.append(np.exp(slope))
                SE.append(np.exp(slope+slope_standard_error)-np.exp(slope))
                p.append(slope_pvalue)
                cor.append(spearmanr(T_months, Reco_months)[0])
    
    result = np.array([site, igbp, year_list, Q10, SE, p, cor])
    result_pd = pd.DataFrame(result.T)
    save_path = './Q10_site_year_nighttime.csv'
    result_pd.to_csv(save_path, index= 'a', header= False)

In [ ]:
path = 'D:Data/Fluxnet/All_hourly_data/'

site = ['site']
year_list = ['year']
T_mean = ['Tm']


for csv_file in os.listdir(path):

    data = pd.read_csv(path+csv_file)
    year = (data['TIMESTAMP_START']/100000000).astype(int)
    month = np.round(data['TIMESTAMP_START']/1000000).astype(int)%100
    unique_values = np.unique(year)

    for i in range(len(unique_values)):
        
        Ti = data['TA_F_MDS'].values[(year==unique_values[i])]
        T_QCi = data['TA_F_MDS_QC'].values[(year==unique_values[i])]
        
        Ti_1 = Ti[(Ti!=-9999) & (T_QCi == 0)]
        if len(Ti_1)>0.9*len(Ti):
            site.append(csv_file[4:10])
            year_list.append(unique_values[i])
            T_mean.append(np.mean(Ti_1))
        result = np.array([site, year_list, T_mean])
        result_pd = pd.DataFrame(result.T)
        save_path = './Site_mean_T.csv'
        result_pd.to_csv(save_path, index= 'a', header= False)

In [11]:
path1 = './Q10_site_year_nighttime.csv'
path2 = './Site_climate.csv'

data1 = pd.read_csv(path1)
data2 = pd.read_csv(path2)

site = ['site']
climate = ['climate']
cor = ['cor']

m,n = data1.shape

for i in range(m):
    if data1['site'][i] in data2['site'].values:
        print(1)
        site.append(data1['site'][i])
        climate.append( data2['climate'].values[(data2['site'] == data1['site'][i])][0][0] )
        cor.append(data1['cor'][i])
        
result = np.array([site, climate, cor])
result_pd = pd.DataFrame(result.T)
save_path = './Cor_climate.csv'
result_pd.to_csv(save_path, index= 'a', header= False)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
